# Colab TTS WebSocket Server (GPU)

Steps:
1. Switch runtime to GPU.
2. Install deps (uv, piper-tts, onnxruntime-gpu, fastapi, uvicorn, websockets, numpy, pydantic).
3. Download a Piper voice.
4. Start the FastAPI websocket server on port 8000.
5. (Optional) Expose with ngrok or run a Colab client in another cell.


In [ ]:
%%bash
nvidia-smi || true


In [ ]:
%%bash
pip install -U pip
pip install uv piper-tts fastapi uvicorn websockets numpy pydantic onnxruntime-gpu


In [ ]:
%%bash
set -euo pipefail
python -m pip install -U piper-tts
mkdir -p /content/voices
python -m piper.download_voices --download-dir /content/voices en_US-lessac-medium
ls -lh /content/voices


In [ ]:
%%bash
if [ ! -d /content/sigiq_takehome ]; then
  echo "Upload your repo directory to /content/sigiq_takehome first" && exit 1
fi
cd /content/sigiq_takehome


In [ ]:
%%bash
# start server (background)
cd /content/sigiq_takehome
export TTS_BACKEND=piper
export PIPER_VOICE=/content/voices/en_US-lessac-medium.onnx
export PIPER_USE_CUDA=1
# optional: disable aligner for lowest latency
unset ALIGNER
nohup uvicorn app.server:app --host 0.0.0.0 --port 8000 > server.log 2>&1 &
pgrep -fl uvicorn


In [ ]:
# Tail server logs to verify startup
import subprocess, time, os, textwrap, sys
log_path = "/content/sigiq_takehome/server.log"
if os.path.exists(log_path):
    with open(log_path) as f:
        print(f.read().splitlines()[-20:])
else:
    print("Log not found; run the server cell first.")


In [ ]:
# Simple Colab client test against the server
import asyncio, json, base64, wave
import websockets

async def main():
    uri = "ws://localhost:8000/tts"
    text = "Hello from Colab GPU streaming TTS."
    pcm_chunks = []
    async with websockets.connect(uri, max_size=None) as ws:
        await ws.send(json.dumps({"text": " ", "flush": False}))
        await ws.send(json.dumps({"text": text, "flush": False}))
        await ws.send(json.dumps({"text": "", "flush": True}))
        await ws.send(json.dumps({"text": "", "flush": False}))
        try:
            while True:
                msg = await ws.recv()
                if isinstance(msg, bytes):
                    msg = msg.decode()
                payload = json.loads(msg)
                audio_b64 = payload.get("audio", "")
                if audio_b64:
                    pcm_chunks.append(base64.b64decode(audio_b64))
        except websockets.ConnectionClosed:
            pass
    pcm = b"".join(pcm_chunks)
    with wave.open("/content/output.wav", "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(44100)
        wf.writeframes(pcm)
    print("Wrote /content/output.wav, bytes:", len(pcm))

asyncio.run(main())
